In [1]:
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.ERROR)
import threading
kite = KiteConnect(api_key="w19o0chuo929jxkp")
import joblib
kite = joblib.load('kitefile.p')
kite.login_url()

'https://kite.trade/connect/login?api_key=w19o0chuo929jxkp&v=3'

In [ ]:
mytoken = input('Enter Token Here : ')

data = kite.generate_session(mytoken, 
                             api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
kite.set_access_token(data["access_token"])
import joblib
joblib.dump(kite,'kitefile.p')

In [ ]:
import act

In [ ]:
kite.orders()[-1]

In [ ]:
kite.positions()['net']

In [ ]:
kite.holdings()

In [ ]:
kite.margins(segment = 'equity')['available']['live_balance']

In [2]:
#####################
pivot_price = 73.4000
#####################
fullquantity = 45
per_order_quant = 10
#####################
initial_difference = 0.0025
difference = 0.0400
#####################

stopbuy = False
stopsell = False

base=0.0025
max_no_of_order = int(fullquantity / per_order_quant)
total_parts = 2*max_no_of_order

print()
print('pivot_price : '+str(pivot_price))
print()
for i in range(1,max_no_of_order+1):
    globals()['p'+str(i)+'b'] = pivot_price - (initial_difference + i* difference)
    globals()['p'+str(i)+'b'] = round(base * round(globals()['p'+str(i)+'b'] /base),4)
    print('p'+str(i)+'b : '+str(globals()['p'+str(i)+'b']))

print()
for i in range(1,max_no_of_order+1):
    globals()['p'+str(i)+'s'] = pivot_price + (initial_difference + i* difference)
    globals()['p'+str(i)+'s'] = round(base * round(globals()['p'+str(i)+'s'] /base),4)
    print('p'+str(i)+'s : '+str(globals()['p'+str(i)+'s']))

print()
for i in range(1,max_no_of_order+1):
    globals()['p'+str(i)+'q'] = per_order_quant*i
    print('p'+str(i)+'q : '+str(globals()['p'+str(i)+'q']))


pivot_price : 73.4

p1b : 73.3575
p2b : 73.3175
p3b : 73.2775
p4b : 73.2375

p1s : 73.4425
p2s : 73.4825
p3s : 73.5225
p4s : 73.5625

p1q : 10
p2q : 20
p3q : 30
p4q : 40


In [3]:
myquantity = 0
def getquant():
    global myquantity
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']=='USDINR20OCTFUT'):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))

getquant()

def placeneworder(quantdiff):
    global stopbuy
    global stopsell
    if(quantdiff>0 and stopbuy == False):
        try:
                order_id= kite.place_order(tradingsymbol="USDINR20OCTFUT",
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_NRML)
                getquant()
                stopsell = False
        except Exception as e:
            stopbuy = False
            print(e)
            getquant()
    if(quantdiff<0 and stopsell == False):
        try:
                order_id= kite.place_order(tradingsymbol="USDINR20OCTFUT",
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_NRML)
                getquant()
                stopbuy = False
        except Exception as e:
            stopsell = True
            print(e)
            getquant()

        
def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff)
    return myquantity


last_price = pivot_price

def checkpivot(ltps):
    global last_price
    trade_quant = 0
    
    if(ltps > (globals()['p'+str(max_no_of_order)+'s']+(2*difference) ) or ltps < (globals()['p'+str(max_no_of_order)+'s']-(2*difference) )):
        ckqnt(pivot_price,0)
    
    elif ((ltps>=(pivot_price-0.0025)) and (ltps<=(pivot_price+0.0025))):
        ckqnt(pivot_price,0)
        print('in pivot range')

    else:
        for i in range(1,int(total_parts/2)+1):

            if(ltps<pivot_price):
                trade_quant = globals()['p'+str(i)+'q']
            elif(ltps>pivot_price):
                trade_quant = -globals()['p'+str(i)+'q']

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps < last_price):
                ckqnt(globals()['p'+str(i)+'b'],trade_quant)
                print('==========' +'p'+str(i)+'b @ ' +str(ltps) + '==============' )
                last_price=ltps

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps > last_price):
                ckqnt(globals()['p'+str(i)+'s'],trade_quant)
                print('==========' +'p'+str(i)+'s @ ' +str(ltps) + '==============' )
                last_price=ltps

My Quantity : -45


In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#              START TRADING             !
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
kws = KiteTicker("w19o0chuo929jxkp", "lUL4kr5aYIlHBwNMldIKwLy8zbNpS5Ev")

def on_ticks(ws, ticks):
    checkpivot(ticks[0]['last_price'])
    #print(type(ticks[0]['last_price']))
    #print('==========================')
    #print( 'LTP : ' + str(ticks[0]['last_price']))
    #print('==========================')
    #print()

def on_connect(ws, response):
    ws.subscribe([690691])
    ws.set_mode(ws.MODE_FULL, [690691])

def on_close(ws, code, reason):
    ws.stop()

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()